Holli Knight
STAT 760

CIFAR10_NeuralNetwork

In [ ]:
from torchvision.datasets import CIFAR10
df = CIFAR10('data', train=True, download=True, transform=_tasks)

In [102]:
from torchvision import transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch 
import torch.nn as nn

## create training and validation split 
split = int(0.8 * len(df))
index_list = list(range(len(df)))
X_train, X_test = index_list[:split], index_list[split:]

## create training and validation sampler objects
tr_sampler = SubsetRandomSampler(X_train)
val_sampler = SubsetRandomSampler(X_test)

## create iterator objects for train and valid datasets
trainloader = DataLoader(df, batch_size=256, sampler=tr_sampler)
validloader = DataLoader(df, batch_size=256, sampler=val_sampler)

In [103]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.linear1 = nn.Linear(1024, 512)
        self.linear2 = nn.Linear(512, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 1024) 
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x

model = Model()

In [138]:
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay= 1e-7, momentum = 0.7, nesterov = True)

In [139]:
for epoch in range(1, 9999):
    train_loss, valid_loss = [], []

In [140]:
model.train()
for data, target in trainloader:
    optimizer.zero_grad()
    output = model(data)
    loss = loss_function(output, target)
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item()) 
        

model.eval()
for data, target in validloader:
    output = model(data)
    loss = loss_function(output, target)
    valid_loss.append(loss.item())

In [141]:
dataiter = iter(validloader)
data, labels = dataiter.next()
output = model(data)

In [142]:
import numpy as np

_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy())

In [143]:
print ("Actual:", labels[:10])
print ("Predicted:", preds[:10])

Actual: tensor([2, 9, 3, 2, 2, 0, 0, 4, 5, 8])
Predicted: [2 9 8 6 7 0 2 9 6 8]


In [144]:
from sklearn.metrics import classification_report 
results = classification_report(labels, preds) 
print(results)

             precision    recall  f1-score   support

          0       0.71      0.65      0.68        31
          1       0.53      0.41      0.46        22
          2       0.38      0.20      0.26        30
          3       0.67      0.10      0.17        20
          4       0.50      0.28      0.36        25
          5       0.75      0.09      0.15        35
          6       0.35      0.71      0.47        24
          7       0.32      0.81      0.46        26
          8       0.44      0.64      0.52        28
          9       0.30      0.40      0.34        15

avg / total       0.51      0.43      0.39       256

